#  Deep learning-based segmentation using CERR-Octave


### Output structures:

*   Masseters (left, right)
*   Medial pterygoid (left, right)
*   Larynx
*   Constrictor muscle

---

### Pull pretrained models and python wrappers

In [1]:
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/CT_SwallowingAndChewing_DeepLabV3/releases/latest \
| grep "tag_name" \
| awk '{print "https://github.com/cerr/CT_SwallowingAndChewing_DeepLabV3/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o CT_swallow_chew.zip $LOCATION

unzip CT_swallow_chew.zip -d CT_SwallowingAndChewing_DeepLabV3
releaseDir=$(ls CT_SwallowingAndChewing_DeepLabV3)

mv CT_SwallowingAndChewing_DeepLabV3/*/* CT_SwallowingAndChewing_DeepLabV3/
rm CT_swallow_chew.zip
rm -r "CT_SwallowingAndChewing_DeepLabV3/$releaseDir"

Archive:  CT_swallow_chew.zip
3b14430171cc9a4f06f086a405a408e2565f36da
   creating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/.gitattributes  
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/README.md  
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/environment.yml  
   creating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/model/
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/model/CM_Ax_model.pth.tar  
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/model/CM_Cor_model.pth.tar  
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/model/CM_Sag_model.pth.tar  
  inflating: CT_SwallowingAndChewing_DeepLabV3/CT_SwallowingAndChewing_DeepLabV3-1.1/model/Larynx_Ax_model.p

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   141  100   141    0     0    770      0 --:--:-- --:--:-- --:--:--   770
100 4504M    0 4504M    0     0  15.2M      0 --:--:--  0:04:55 --:--:-- 41.4M


### Install Octave 5.2

In [2]:
! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps
! pip install octave_kernel
! pip install oct2py

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [43.0 kB]
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubu

In [3]:
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/releases/latest \
| grep "tag_name" \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

Archive:  octavecolab.zip
7f7b537619f90d32bdd555de6762743e1f85ef36
   creating: octavecolab/octave-colab-1.0/
  inflating: octavecolab/octave-colab-1.0/.gitattributes  
 extracting: octavecolab/octave-colab-1.0/README.md  
  inflating: octavecolab/octave-colab-1.0/build_octave5_2.ipynb  
  inflating: octavecolab/octave-colab-1.0/conda_setup.ipynb  
  inflating: octavecolab/octave-colab-1.0/npm_box_setup.ipynb  
  inflating: octavecolab/octave-colab-1.0/octave5_2_0_deploy.ipynb  
  inflating: octavecolab/octave-colab-1.0/octave5_2_0_deploy_X11.ipynb  
  inflating: octavecolab/octave-colab-1.0/octavecolab.tar.gz  
octavecolab/
octavecolab/lib/
octavecolab/lib/octave/
octavecolab/lib/octave/site/
octavecolab/lib/octave/site/oct/
octavecolab/lib/octave/site/oct/x86_64-pc-linux-gnu/
octavecolab/lib/octave/site/oct/api-v53/
octavecolab/lib/octave/site/oct/api-v53/x86_64-pc-linux-gnu/
octavecolab/lib/octave/5.2.0/
octavecolab/lib/octave/5.2.0/liboctave.so.7.0.1
octavecolab/lib/octave/5.2.0/oc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   120  100   120    0     0    641      0 --:--:-- --:--:-- --:--:--   641
100  187M    0  187M    0     0  10.4M      0 --:--:--  0:00:17 --:--:-- 35.3M


In [4]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

In [5]:
%load_ext oct2py.ipython

!apt install octave liboctave-dev
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  info install-info liboctave4 libqt5opengl5 octave-common octave-doc
Suggested packages:
  texinfo-doc-nonfree www-browser | pdf-viewer
The following NEW packages will be installed:
  info install-info liboctave-dev liboctave4 libqt5opengl5 octave
  octave-common octave-doc
0 upgraded, 8 newly installed, 0 to remove and 77 not upgraded.
Need to get 17.3 MB of archives.
After this operation, 73.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 install-info amd64 6.5.0.dfsg.1-2 [55.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 info amd64 6.5.0.dfsg.1-2 [104 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 liboctave4 amd64 4.2

### Pull CERR-Octave

In [6]:
%cd /content
!git clone https://github.com/cerr/CERR.git
%cd /content/CERR
!git checkout octave_dev
%cd ..

/content
Cloning into 'CERR'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 20609 (delta 27), reused 40 (delta 19), pack-reused 20537
Receiving objects: 100% (20609/20609), 357.49 MiB | 31.13 MiB/s, done.
Resolving deltas: 100% (14075/14075), done.
Checking out files: 100% (3489/3489), done.
/content/CERR
Branch 'octave_dev' set up to track remote branch 'octave_dev' from 'origin'.
Switched to a new branch 'octave_dev'
/content


### Create conda environment with dependencies

In [7]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

--2020-09-11 18:07:36--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   270MB/s    in 0.3s    

2020-09-11 18:07:37 (270 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [8]:
!conda env create -f CT_SwallowingAndChewing_DeepLabV3/environment.yml

Solving environment: / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



python-3.7.3         | 36.7 MB   | : 100% 1.0/1 [00:04<00:00,  4.78s/it]               
sqlite-3.27.2        | 1.9 MB    | : 100% 1.0/1 [00:00<00:00,  3.47it/s]
certifi-2020.6.20    | 151 KB    | : 100% 1.0/1 [00:00<00:00, 20.01it/s]
_openmp_mutex-4.5    | 22 KB     | : 100% 1.0/1 [00:00<00:00, 36.35it/s]
wheel-0.33.1         | 34 KB     | : 100% 1.0/1 [00:00<00:00, 26.50it/s]
tk-8.6.8             | 3.1 MB    | : 100% 1.0/1 [00:00<00:00,  2.00it/s]
_libgcc_mutex-0.1    | 3 KB      | : 100% 1.0/1 [00:00<00:00, 23.70it/s]
openssl-1.1.1g       | 2.1 MB    | : 100% 1.0/1 [00:00<00:00,  3.17it/s]
python_abi-3.7       | 4 KB      | : 100% 1.0/1 [00:00<00:00, 34.86it/s]
zlib-1.2.11          | 106 KB    | : 100% 1.0/1 [00:00<00:00, 25.24it/s]
ca-certificates

### Run inference

In [9]:
from oct2py import octave

octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.addpath(octave.genpath('/content/CERR'))

'.:/content/octavecolab/share/octave/packages/statistics-1.4.2/base:/content/octavecolab/share/octave/packages/statistics-1.4.2/distributions:/content/octavecolab/share/octave/packages/statistics-1.4.2/models:/content/octavecolab/share/octave/packages/statistics-1.4.2/tests:/content/octavecolab/share/octave/packages/statistics-1.4.2:/content/octavecolab/share/octave/packages/io-2.6.1:/content/octavecolab/lib/octave/packages/io-2.6.1/x86_64-pc-linux-gnu-api-v53:/content/octavecolab/share/octave/packages/image-2.12.0:/content/octavecolab/lib/octave/packages/image-2.12.0/x86_64-pc-linux-gnu-api-v53:/usr/local/lib/python3.6/dist-packages/oct2py:/usr/local/lib/python3.6/dist-packages/octave_kernel:/content/octavecolab/lib/octave/5.2.0/site/oct/x86_64-pc-linux-gnu:/content/octavecolab/lib/octave/site/oct/api-v53/x86_64-pc-linux-gnu:/content/octavecolab/lib/octave/site/oct/x86_64-pc-linux-gnu:/content/octavecolab/share/octave/5.2.0/site/m:/content/octavecolab/share/octave/site/api-v53/m:/cont

In [10]:
%%capture

sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/' + \
              'CERR_plans/head_neck_ex1_20may03.mat.bz2'
%octave_push sampleData
planC = octave.loadPlanC(sampleData, octave.tempdir())
planC = octave.updatePlanFields(planC)
planC = octave.quality_assure_planC(sampleData, planC)

algorithm = 'CT_ChewingStructures_DeepLabV3';
sessionPath = '/content/temp';
functionName = '/content/CT_SwallowingAndChewing_DeepLabV3/' + \
               'model_wrapper/CT_ChewingStructures_DeepLabV3/chewing_main.py';
condaEnvName = 'swChewEnv'

planC = octave.runSegForPlanCInCondaEnv(planC,sessionPath,algorithm, \
                                        functionName,[],condaEnvName);

CERR>>  Decompressing head_neck_ex1_20may03.mat.bz2...

ans = 0
CERR>>  Loading head_neck_ex1_20may03.mat.bz2...
CERR>>  Loaded head_neck_ex1_20may03.mat.bz2...
ans = 1
ans = 1
ans = 1
ans = 1
ans = 1
ans = 1
userInS =

  scalar structure containing the fields:

    strNameToLabelMap =

      1x4 struct array containing the fields:

        structureName
        value

    inputFileType = CERR
    view =
    {
      [1,1] = axial
      [1,2] = sagittal
      [1,3] = coronal
    }

    crop =

      scalar structure containing the fields:

        method = crop_for_chewing_structs
        params =

          scalar structure containing the fields:

            structureName: 1x7 sq_string
            headSizeLimitCm: 1x1 scalar
            saveStrToPlanCFlag: 1x1 scalar

        operator =

    resize =

      scalar structure containing the fields:

        method = none
        size =

           320   320


    channels =

      1x3 struct array containing the fields:

        imageT

### Display segmentations

In [20]:
from oct2py import octave
%octave_push planC